对每个H*M*S*文件检查，是否对应的视频帧数和时间帧数相等，以时间戳为准，删除多余的帧

In [2]:
import cv2
import os,sys,glob
import re
import pandas as pd
import numpy as np

In [3]:
HMSfilepathes = glob.glob(r"D:\miniscope\20200801\206550\H*")
# HMSfilepathes = [i for i in HMSfilepathes if "20200720" not in i]
HMSfilepathes

['D:\\miniscope\\20200801\\206550\\H16_M12_S15',
 'D:\\miniscope\\20200801\\206550\\H16_M17_S16',
 'D:\\miniscope\\20200801\\206550\\H16_M1_S7',
 'D:\\miniscope\\20200801\\206550\\H16_M22_S17',
 'D:\\miniscope\\20200801\\206550\\H16_M27_S29',
 'D:\\miniscope\\20200801\\206550\\H16_M7_S15']

In [4]:
def sort_key(s):
    """
    fit for video recorded by miniscope.exe
    sort fixed pattern: YYYYMMDD/H*M*S*/msCam*.avi
    """
    if s:
        try:
            ms = re.findall('msCam(\d+)',s)[0]
        except:
            ms = -1
        return [int(ms)]
    
def len_of_ts(tsFile):
    datatemp=pd.read_csv(tsFile[0],sep = "\t", header = 0)
    if np.abs(datatemp['sysClock'][0])>datatemp['sysClock'][1]:
        value = datatemp['sysClock'][1]-13 # 用第2帧的时间减去13，13是大约的一个值
        if value < 0:
            datatemp['sysClock'][0]=0
        else:
            datatemp['sysClock'][0]=value

    ts = datatemp['sysClock'].values
    
    return len(ts),ts

def len_of_ms(msFileList):
    lens=[]
    for msFile in msFileList:
        lens.append(int(cv2.VideoCapture(msFile).get(7)))
    return sum(lens)

In [7]:
bug_files=[]
for file in HMSfilepathes:
    print(file,end="<-->")
    msFileList = glob.glob(os.path.join(file,"msCam*.avi"))
    msFileList.sort(key=sort_key)
    tsFile = glob.glob(os.path.join(file,"timestamp.dat"))
    ms_lens=len_of_ms(msFileList)
    ts_lens=len_of_ts(tsFile)
    if ms_lens!=ts_lens[0]:
        bug_files.append((file,ms_lens,ts_lens))
    else:
        print(ms_lens,ts_lens)


D:\miniscope\20200801\206550\H16_M12_S15<-->9008 (9008, array([    52,     65,     77, ..., 299976, 300004, 300038], dtype=int64))
D:\miniscope\20200801\206550\H16_M17_S16<-->9007 (9007, array([    11,     24,     40, ..., 299909, 299946, 299977], dtype=int64))
D:\miniscope\20200801\206550\H16_M1_S7<-->9008 (9008, array([     9,     22,     56, ..., 299957, 299990, 300024], dtype=int64))
D:\miniscope\20200801\206550\H16_M22_S17<-->1819 (1819, array([   13,    26,    60, ..., 60479, 60513, 60548], dtype=int64))
D:\miniscope\20200801\206550\H16_M27_S29<-->22731 (22731, array([    17,     30,     50, ..., 757753, 757787, 757820], dtype=int64))
D:\miniscope\20200801\206550\H16_M7_S15<-->9010 (9010, array([    14,     25,     53, ..., 300025, 300056, 300089], dtype=int64))


In [6]:
bug_files

[]

In [8]:
import pickle

In [19]:
with open(r"\\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206550\20200816_102733_30fps_20200801\ms_ts.pkl","rb") as f:
    result = pickle.load(f)
sum([len(i) for i in result])

60583

In [17]:
cv2.VideoCapture(r"\\10.10.47.163\Data_archive\qiushou\miniscope_result\Results_206550\20200816_102733_30fps_20200801\msCam_concat.avi").get(7)

60584.0